In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.devices.experimental import TestDevicePythonSim, PlainNumpySimulator

In [20]:
dev = TestDevicePythonSim()

## The Preprocessor

Suppose we create a `QuantumScript` with operations not natively supported by the device.

In [3]:
weights = np.array([[0.1, -2.1, 1.4]])
ops = [qml.RandomLayers(weights=weights, wires=range(2))]

qs1 = qml.tape.QuantumScript(ops, [qml.state()])

When we try to execute that script, things will fall over and die with unhelpful error messages.

In [4]:
dev.execute(qs1)

MatrixUndefinedError: 

So we first want to preprocess and validate the qscript.  Once we processed it, we have something
the device can execute.

In [ ]:
processed_qs1 = dev.preprocess(qs1)
dev.execute(processed_qs1)

Preprocessing can include more than just "I support this" and "I don't support this".

It could also include compilation steps, like running `qml.simplify`.

In [ ]:
qs3 = qml.tape.QuantumScript([qml.PauliX(0) ** 0.5], [qml.state()])
dev.preprocess(qs3).circuit

If it *cant* convert the `QuantumScript` into something executable, it would raise an error
at this point, before any executions begin.

In [ ]:
qs2 = qml.tape.QuantumScript([qml.S(i) for i in range(55)], [qml.state()])
dev.preprocess(qs2)

# Derivatives

In [ ]:
a = [0.1, 0.2, 0.3]
ops = [qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
measurement = qml.expval(qml.PauliX(1))
qs = qml.tape.QuantumScript(ops, [measurement])

dev.gradient(qs)

## The Simulator

In this prototype, the "Simulator" is separate from the device interface.

In [ ]:
sim = PlainNumpySimulator()

Since we are not mixing the device interface with the simulator design, we could make the
`PlainNumpySimulator` a user-facing object in its own right.

This would make it more easier the explain and play with the simulator internals. For example, we can calculate that adjoint jacobian without resorting to private methods.

In [ ]:
a = [0.1, 0.2, 0.3]
ops = [qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
measurement = qml.expval(qml.PauliX(1))

state = sim.create_zeroes_state(2)
for op in ops:
    state = sim.apply_operation(state, op)
bra = sim.apply_operation(state, measurement.obs)
ket = state

grads = []
for op in reversed(ops):
    adj_op = qml.adjoint(op)
    ket = sim.apply_operation(ket, adj_op)

    if op.num_params != 0:
        dU = qml.operation.operation_derivative(op)
        ket_temp = sim.apply_matrix(ket, dU, op.wires)
        dM = 2 * np.real(np.vdot(bra, ket_temp))
        grads.append(dM)

    bra = sim.apply_operation(bra, adj_op)

grads = grads[::-1]
print(grads)


# Integration

In [21]:
qml.enable_return()

In [30]:
dev.batch_execute = dev.execute

dev.batch_transform = dev.preprocess
dev.expand_fn = lambda circuit, max_expansion: circuit
dev.shots = None
dev._shot_vector = []
dev.shot_vector = None

In [31]:
@qml.qnode(dev, diff_method="device")
def circuit(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))
a
x = qml.numpy.array([1.2, 2.3, 3.4])


In [32]:
circuit(x)

[-0.2414302829326092]

In [33]:
qml.grad(circuit)(x)

array([0.67195027, 0.23341436, 0.06905029])

In [ ]:
@qml.qnode(dev, diff_method=qml.gradients.param_shift)
def circuit(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

x = qml.numpy.array([1.2, 2.3, 3.4])
qml.grad(circuit)(x)